In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [ ]:
df = pd.read_csv("client_data (1).csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.tail()

In [ ]:
#df.columns.value_counts()

In [ ]:
df1 = pd.read_csv("price_data (1).csv")
df1

In [ ]:
df1.shape

In [ ]:
final_df = df.join(df1.set_index('id'), on= "id", how='left')
final_df

In [ ]:
#df['id'].duplicated()

In [ ]:
final_df.isna().sum()

In [ ]:
#df['channel_sales'].value_counts()

In [ ]:
#df['origin_up'].value_counts()

In [ ]:
final_df.dropna(how = "all",inplace = True)

In [ ]:
final_df.shape

In [ ]:
final_df.columns

In [ ]:
final_df.dropna()

In [ ]:
plt.figure(figsize =(20,8), dpi=200)
sns.heatmap(final_df.corr(),cmap ="PiYG", annot = True)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

#variables for which VIF to be calculated
X = final_df[['cons_12m', 'cons_gas_12m', 'cons_last_month',
       'forecast_cons_12m', 'forecast_cons_year', 'forecast_discount_energy',
       'forecast_meter_rent_12m', 'forecast_price_energy_off_peak',
       'forecast_price_energy_peak', 'forecast_price_pow_off_peak',
       'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act',
       'net_margin', 'num_years_antig', 'pow_max', 'churn', 'price_off_peak_var', 'price_peak_var',
       'price_mid_peak_var', 'price_off_peak_fix', 'price_peak_fix',
       'price_mid_peak_fix']]

#converting into dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns

#VIF Calculation
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]

print(vif_data)

In [ ]:
final_df.info()

In [ ]:
final_df.columns

In [ ]:
final_df1= final_df[['id', 'channel_sales', 'cons_12m', 'cons_gas_12m', 'cons_last_month',
       'date_activ', 'date_end', 'date_modif_prod', 'date_renewal',
      'forecast_cons_year', 'forecast_discount_energy',
       'forecast_meter_rent_12m', 'has_gas',
       'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act',
       'net_margin', 'num_years_antig', 'origin_up', 'pow_max', 'churn',
       'price_date', 'price_off_peak_var', 'price_peak_var',
   'price_off_peak_fix', 'price_peak_fix']]

In [ ]:
final_df1

In [ ]:
final_df1.info()

In [ ]:
cat_cols = ['channel_sales','date_activ','date_end','date_modif_prod','date_renewal','has_gas','origin_up','price_date']

In [ ]:
#final_df1['date_activ'] = final_df1.date_activ.astype(str)
#final_df1['date_activ']

In [ ]:
from scipy.stats import chi2_contingency

def get_chi_sq(row, column, df):

    #plotting the graph for each independent variable with target variable
    fig = plt.figure()
    ax = fig.add_subplot(111)
    sns.countplot(data=final_df1, x=row, hue=column)
    plt.xlabel(row)
    plt.ylabel('Count')
    plt.show()
    #doing this to keep from opening too many figs at once
    plt.close(fig);

    #creating cross tab to calculate chi square values
    xtab = pd.crosstab(final_df1[row], final_df1[column], margins = False)
    print('observed data:\n', xtab)

    #Chi-square contingency table
    chi2, p_value, dof, expected = chi2_contingency(xtab, correction = False)
    print('expected data:\n', expected)
    print('chi-squared value: {:.4f} for {:.0f} dof; p-value = {:.4f}\n'.format(chi2, dof, p_value))

In [ ]:
#for every category column, doing the Chi-sq test
for cat_var in cat_cols:
  get_chi_sq(cat_var, 'churn', final_df1)      #pricedate remove

In [ ]:
final_df1.drop("price_date", axis =1, inplace = True)

In [ ]:
final_df1.shape

In [ ]:
final_df1.columns

In [ ]:
final_df1.shape

In [ ]:
a = ['channel_sales','origin_up']

In [ ]:
l = LabelEncoder()
for i in a:
    final_df1[i]=l.fit_transform(final_df1[i])
final_df1

In [ ]:
final_df1['origin_up'].unique()

In [ ]:
dummies = pd.get_dummies(final_df1.has_gas, prefix = 'hg')
dummies

In [ ]:
con_cat = pd.concat([final_df1, dummies], axis =1)
con_cat

In [ ]:
final = con_cat.drop(columns=["has_gas"])
final

In [ ]:
h_g = ['hg_f','hg_t']

In [ ]:
from scipy.stats import chi2_contingency

def get_chi_sq(row, column, final):

    #plotting the graph for each independent variable with target variable
    fig = plt.figure()
    ax = fig.add_subplot(111)
    sns.countplot(data=final, x=row, hue=column)
    plt.xlabel(row)
    plt.ylabel('Count')
    plt.show()
    #doing this to keep from opening too many figs at once
    plt.close(fig);

    #creating cross tab to calculate chi square values
    xtab = pd.crosstab(final[row], final[column], margins = False)
    print('observed data:\n', xtab)

    #Chi-square contingency table
    chi2, p_value, dof, expected = chi2_contingency(xtab, correction = False)
    print('expected data:\n', expected)
    print('chi-squared value: {:.4f} for {:.0f} dof; p-value = {:.4f}\n'.format(chi2, dof, p_value))

In [ ]:
#for every category column, doing the Chi-sq test
for var1 in h_g:
  get_chi_sq(var1, 'churn', final)

In [ ]:
final.drop('id', inplace = True, axis = 1)

In [ ]:
final

In [ ]:
final['date_activ'] = pd.to_datetime(final.date_activ)
final['date_activ']

In [ ]:
final['date_end'] = pd.to_datetime(final.date_end)
final['date_end']

In [ ]:
final['date_modif_prod'] = pd.to_datetime(final.date_modif_prod)
final['date_modif_prod']

In [ ]:
final['date_renewal'] = pd.to_datetime(final.date_renewal)
final['date_renewal']

In [ ]:
final

In [ ]:
final.info()

In [ ]:
#final['Diff_activ_end'] = final.apply(lambda row: (row['date_end'] - row['date_activ']).days, axis=1)
#final['Diff_activ_end']= final['Diff_activ_end'].astype(int)

In [ ]:
final['diff_activ_years'] = (final['date_end'] - final['date_activ']) / np.timedelta64(1, 'Y')
final['diff_activ_years']= final['diff_activ_years'].astype(int)

In [ ]:
final

In [ ]:
final['diff_activ_years'].unique()

In [ ]:
final['diff_renewal_years'] = (final['date_end'] - final['date_renewal']) / np.timedelta64(1, 'Y')
final['diff_renewal_years']= final['diff_renewal_years'].astype(int)

In [ ]:
final

In [ ]:
final['diff_renewal_years'].unique()

In [ ]:
#final[['month_activ', 'year_activ','day_activ']] = final['date_activ'].str.split('-', 1, expand=True)
#final

In [ ]:
#final['day_activ'] = final['date_activ'].dt.day
#final['month_activ'] = final['date_activ'].dt.month
#final['year_activ'] = final['date_activ'].dt.year
#final

In [ ]:
#final['day_end'] = final['date_end'].dt.day
#final['month_end'] = final['date_end'].dt.month
#final['year_end'] = final['date_end'].dt.year
#final

In [ ]:
#final['day_modif_prod'] = final['date_modif_prod'].dt.day
#final['month_modif_prod'] = final['date_modif_prod'].dt.month
#final['year_modif_prod'] = final['date_modif_prod'].dt.year
#final

In [ ]:
#final['day_renewal'] = final['date_renewal'].dt.day
#final['month_renewal'] = final['date_renewal'].dt.month
#final['year_renewal'] = final['date_renewal'].dt.year
#final

In [ ]:
final.drop(columns=['date_activ',	'date_end',	'date_renewal'], inplace = True, axis = 1)


In [ ]:
final

In [ ]:
final.info()

In [ ]:
#sns.scatterplot(data=final, x="date_modif_prod ", y="churn")
#plt.show()

In [ ]:
final.drop("date_modif_prod", inplace = True, axis = 1)

In [ ]:
final

In [ ]:
X = final.drop('churn', axis = 1)
y = final['churn']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
sc = StandardScaler()
X_Train = sc.fit_transform(X_train)
X_Test = sc.transform(X_test)

In [ ]:
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 1, 10, 100]
}

In [ ]:
from sklearn.model_selection import GridSearchCV
lr = LogisticRegression()
clf = GridSearchCV(lr, param_grid, cv=5,scoring = 'accuracy')

In [ ]:
clf.fit(X_Train, y_train)

In [ ]:
print('Best Parameters:',clf.best_params_,end='\n\n')
print('Best Score:',clf.best_score_)

In [ ]:
clf.fit(X_Test, y_test)

In [ ]:
print('Best Parameters:',clf.best_params_,end='\n\n')
print('Best Score:',clf.best_score_)

In [ ]:
lr = LogisticRegression(C= 100)
lr.fit(X_Train, y_train)

In [ ]:
lr_acc = accuracy_score(y_train, lr.predict(X_Train))
print(f"Accuracy Score is {lr_acc}")

In [ ]:
lr_acc = accuracy_score(y_test, lr.predict(X_Test))
print(f"Accuracy Score is {lr_acc}")

In [ ]:
dtc = DecisionTreeClassifier()
dtc.fit(X_Train, y_train)

In [ ]:
from sklearn.model_selection import GridSearchCV
dt_hp = DecisionTreeClassifier(random_state=43)

params = {'max_depth':[3,5,7,10,15],
          'min_samples_leaf':[3,5,10,15,20],
          'min_samples_split':[8,10,12,18,20,16],
          'criterion':['gini','entropy']}
GS = GridSearchCV(estimator=dt_hp,param_grid=params,cv=5,n_jobs=-1, verbose=True, scoring='accuracy')

In [ ]:
GS.fit(X_Train, y_train)

In [ ]:
print('Best Parameters:',GS.best_params_,end='\n\n')
print('Best Score:',GS.best_score_)

In [ ]:
GS.fit(X_Test, y_test)

In [ ]:
print('Best Parameters:',GS.best_params_,end='\n\n')
print('Best Score:',GS.best_score_)

In [ ]:
dtc_acc = accuracy_score(y_train, dtc.predict(X_Train))
print(f"Accuracy Score is {dtc_acc}")

In [ ]:
dtc_acc = accuracy_score(y_test, dtc.predict(X_Test))
print(f"Accuracy Score is {dtc_acc}")

In [ ]:
dtc_ps = precision_score(y_test, dtc.predict(X_Test))
print(f"precision_score is {dtc_ps}")                       #from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [ ]:
dtc_recall = recall_score(y_test, dtc.predict(X_Test))
print(f"recall Score is {dtc_recall}")

In [ ]:
dtc_f1s = f1_score(y_test, dtc.predict(X_Test))
print(f"F1 Score is {dtc_f1s}")

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_Train, y_train)

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search
param_grid = {
    'bootstrap': [True],
    'max_depth': [110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 15],
    'min_samples_split': [8, 10, 20],
    'n_estimators': [1000]
}
# Create a based model
rf1 = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf1, param_grid = param_grid,
                          cv = 5, n_jobs = -1, verbose = 2, scoring='accuracy')

In [ ]:
grid_search.fit(X_Train, y_train)

In [ ]:
print('Best Parameters:',grid_search.best_params_,end='\n\n')
print('Best Score:',grid_search.best_score_)

In [ ]:
grid_search.fit(X_Test, y_test)

In [ ]:
print('Best Parameters:',grid_search.best_params_,end='\n\n')
print('Best Score:',grid_search.best_score_)

In [ ]:
rf_acc = accuracy_score(y_train, rf.predict(X_Train))
print(f"Accuracy Score is {rf_acc}")

In [ ]:
rf_acc = accuracy_score(y_test, rf.predict(X_Test))
print(f"Accuracy Score is {rf_acc}")

In [ ]:
rf_recall = recall_score(y_test, rf.predict(X_Test))
print(f"Recall Score is {rf_recall}")

In [ ]:
rf_f1s = f1_score(y_test, rf.predict(X_Test))
print(f"Recall Score is {rf_f1s}")

In [ ]:
importances = rf.feature_importances_
plt.title('Feature Importance')
sorted_indices = np.argsort(importances)[::-1]
plt.bar(range(X_train.shape[1]), importances[sorted_indices], align='center')
plt.xticks(range(X_train.shape[1]), X_train.columns[sorted_indices], rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
gb = GradientBoostingClassifier()
gb.fit(X_Train, y_train)

In [ ]:
gb_acc = accuracy_score(y_test, gb.predict(X_Test))
print(f"Accuracy Score is {gb_acc}")

In [ ]:
gb_acc = accuracy_score(y_train, gb.predict(X_Train))
print(f"Accuracy Score is {gb_acc}")

In [ ]:
#Xgb = XGBClassifier(n_estimators=1000,max_depth=6,
             # min_child_weight=1, num_parallel_tree=None,
             # random_state= 27,learning_rate =0.1,gamma=0.5, colsample_bytree=0.8,cv = 5, scoring = 'accuracy')
#Xgb.fit(X_Train, y_train)

In [ ]:
# Perform hyperparameter tuning using GridSearchCV
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
print('Performing hyperparameter tuning using GridSearchCV...')
params = {'max_depth': [3, 5, 7],
          'learning_rate': [0.1, 0.01, 0.001],
          'subsample': [0.7, 0.8, 0.9],
          'colsample_bytree': [0.7, 0.8, 0.9]}
xgb_model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100, random_state=42)
grid_search = GridSearchCV(estimator=xgb_model, param_grid=params, scoring='accuracy', n_jobs=-1, cv=5)
grid_search.fit(X_Train, y_train)

In [ ]:
print('Best Parameters:',grid_search.best_params_,end='\n\n')
print('Best Score:',grid_search.best_score_)

In [ ]:
grid_search.fit(X_Test, y_test)

In [ ]:
print('Best Parameters:',grid_search.best_params_,end='\n\n')
print('Best Score:',grid_search.best_score_)

In [ ]:
Xgb = XGBClassifier()#n_estimators = 100)
Xgb.fit(X_Train, y_train)

In [ ]:
Xgb_acc = accuracy_score(y_test, Xgb.predict(X_Test))
print(f"Accuracy Score is {Xgb_acc}")

In [ ]:
Xgb_acc = accuracy_score(y_train, Xgb.predict(X_Train))
print(f"Accuracy Score is {Xgb_acc}")

In [ ]:
importances = Xgb.feature_importances_
plt.title('Feature Importance')
sorted_indices = np.argsort(importances)[::-1]
plt.bar(range(X_train.shape[1]), importances[sorted_indices], align='center')
plt.xticks(range(X_train.shape[1]), X_train.columns[sorted_indices], rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
svc=SVC()
svc.fit(X_Train, y_train)

In [ ]:
svc_acc = accuracy_score(y_test, svc.predict(X_Test))
print(f"Accuracy Score is {svc_acc}")

In [ ]:
svc_acc = accuracy_score(y_train, svc.predict(X_Train))
print(f"Accuracy Score is {svc_acc}")

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf','linear']}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,scoring = 'accuracy')

# fitting the model for grid search
grid.fit(X_Train, y_train)

In [ ]:
print('Best Parameters:',grid.best_params_,end='\n\n')
print('Best Score:',grid.best_score_)

In [ ]:
grid.fit(X_Test, y_test)

In [ ]:
print('Best Parameters:',grid.best_params_,end='\n\n')
print('Best Score:',grid.best_score_)